In [1]:
# Setup Spark SQL
# Note if running locally you need the JVM https://www.oracle.com/java/technologies/downloads/
# Consider running in https://colab.research.google.com/
%pip install pyspark

In [15]:

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BibleCounter").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')

In [16]:
# Download the bible as txt
!curl -L "https://www.gutenberg.org/cache/epub/10/pg10.txt" > bible.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4351k  100 4351k    0     0   9.8M      0 --:--:-- --:--:-- --:--:--  9.8M


In [23]:
# Use SparkSQL to list all the words in the bible with their counts sorted descending by count.
# Turn in your code and sample of your results (at least top 20).
# Be sure to ignore case and punctuation and eliminate the blank lines.
import re
import string
from prettytable import PrettyTable
import requests

# Download the Bible text from Gutenberg
url = "https://www.gutenberg.org/cache/epub/10/pg10.txt"
response = requests.get(url)

with open("bible-1.txt", "w", encoding="utf-8") as file:
  file.write(response.text)

# Load the local bible file
lines = sc.textFile("bible-1.txt")

# Find where the actual content starts
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK THE KING JAMES VERSION OF THE BIBLE ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK THE KING JAMES VERSION OF THE BIBLE ***"

def filter_content(line):
    global start_reading
    if start_marker in line:
        start_reading = True
        return False
    if end_marker in line:
        start_reading = False
    return start_reading

start_reading = False

filtered_lines = lines.filter(filter_content)
filtered_lines = filtered_lines.map(lambda line: line.lower())

filtered_lines = filtered_lines.filter(lambda line: len(line.strip()) > 0)

def clean_text(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

words = filtered_lines.map(clean_text).flatMap(lambda line: line.split())
words = words.filter(lambda word: len(word.strip()) > 0)

wordCounts = words.map(lambda w: (w, 1))
wordCounts = wordCounts.reduceByKey(lambda c1, c2: c1 + c2)

top_words = wordCounts.sortBy(lambda x: x[1], False).take(30)

table = PrettyTable()
table.field_names = ["Rank", "Word", "Count"]
table.align["Word"] = "l"
table.align["Count"] = "r"

for i, (word, count) in enumerate(top_words, 1):
    table.add_row([i, word, count])

print(table)

+------+-------+-------+
| Rank | Word  | Count |
+------+-------+-------+
|  1   | the   | 35237 |
|  2   | and   | 29505 |
|  3   | of    | 19663 |
|  4   | to    |  6953 |
|  5   | in    |  6250 |
|  6   | that  |  5661 |
|  7   | he    |  5223 |
|  8   | his   |  4956 |
|  9   | unto  |  4890 |
|  10  | lord  |  4373 |
|  11  | shall |  4099 |
|  12  | for   |  4067 |
|  13  | a     |  3582 |
|  14  | i     |  3353 |
|  15  | him   |  3226 |
|  16  | it    |  3192 |
|  17  | they  |  3172 |
|  18  | thou  |  3081 |
|  19  | with  |  3066 |
|  20  | all   |  3044 |
|  21  | be    |  2972 |
|  22  | them  |  2940 |
|  23  | not   |  2672 |
|  24  | is    |  2535 |
|  25  | said  |  2511 |
|  26  | was   |  2482 |
|  27  | thy   |  2429 |
|  28  | god   |  2133 |
|  29  | which |  2099 |
|  30  | my    |  2082 |
+------+-------+-------+


In [ ]:
# What to try something else?